# **AIM**

## A Python / Java / Spark code which enables:

*   ### The given CSV data to be written, using the distributed storage layout strategy described, to reduce duplicate data
*   ### Retrieval of any record given the record ID from the distributed storage



# **Setup Apache Spark**

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://www-eu.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.0.3-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'/content/spark-3.0.3-bin-hadoop2.7'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2022-03-02 19:08:37--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  18.5MB/s    in 0.7s    

2022-03-02 19:08:38 (18.5 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://1131-34-125-8-227.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50

# **Load and Preprocess Data**

In [ ]:
from pyspark.sql.functions import col, countDistinct, regexp_replace, dense_rank, count, when, isnull, rand

In [ ]:
data = spark.read.csv("/content/drive/MyDrive/Coursera-1.csv", header=True, inferSchema=True)
data.printSchema()

root
 |-- Course Name: string (nullable = true)
 |-- University: string (nullable = true)
 |-- Difficulty Level: string (nullable = true)
 |-- Course Rating: string (nullable = true)
 |-- Course URL: string (nullable = true)
 |-- Course Description: string (nullable = true)
 |-- Skills: string (nullable = true)



In [ ]:
# Display rows
data.show(5)

+--------------------+--------------------+----------------+-------------+--------------------+--------------------+--------------------+
|         Course Name|          University|Difficulty Level|Course Rating|          Course URL|  Course Description|              Skills|
+--------------------+--------------------+----------------+-------------+--------------------+--------------------+--------------------+
|Write A Feature L...|Michigan State Un...|        Beginner|          4.8|https://www.cours...|"Write a Full Len...| writing! You wil...|
|Business Strategy...|Coursera Project ...|        Beginner|          4.8|https://www.cours...|By the end of thi...|Finance  business...|
|Silicon Thin Film...| �cole Polytechnique|        Advanced|          4.1|https://www.cours...|"This course cons...| thin film semico...|
|Finance for Managers|IESE Business School|    Intermediate|          4.8|https://www.cours...|When it comes to ...|accounts receivab...|
|Retrieve Data usi...|Coursera Pro

In [ ]:
# Count no:of rows in data
data.count()

3531

In [ ]:
# Display specific columns
data.select("Course Name", "University", "Difficulty Level").show(5)

+--------------------+--------------------+----------------+
|         Course Name|          University|Difficulty Level|
+--------------------+--------------------+----------------+
|Write A Feature L...|Michigan State Un...|        Beginner|
|Business Strategy...|Coursera Project ...|        Beginner|
|Silicon Thin Film...| �cole Polytechnique|        Advanced|
|Finance for Managers|IESE Business School|    Intermediate|
|Retrieve Data usi...|Coursera Project ...|        Beginner|
+--------------------+--------------------+----------------+
only showing top 5 rows



In [ ]:
# Describe columns
data.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|         Course Name|          University|    Difficulty Level|       Course Rating|          Course URL|  Course Description|              Skills|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  count|                3531|                3526|                3525|                3525|                3524|                3524|                3520|
|   mean|                null|                null|                null|   4.554476744186075|                null|                null|                 2.0|
| stddev|                null|                null|                null| 0.33724464590140496|                null|                null|                 NaN|
|    min|  If you have com...|            cultural| and ae

In [ ]:
data.select("Course Description").distinct().show()

+--------------------+
|  Course Description|
+--------------------+
|Of all the animal...|
|In this course, y...|
|In this 1 hour an...|
|"Do you want to k...|
|This course intro...|
|The proliferation...|
|"Financial Engine...|
|"Interest in medi...|
|In this course yo...|
|This course is fo...|
|Despite the recen...|
|"This course is t...|
|In this 2-hour lo...|
|In order to manag...|
|Meeting another p...|
|In the fourth par...|
|Version Control S...|
|As they tumble th...|
|The second course...|
|Learn the mathema...|
+--------------------+
only showing top 20 rows



In [ ]:
# Preprocess data
data = data.withColumn('Course Name', regexp_replace('Course Name', '[^A-Za-z0-9 ]+', ''))\
    .withColumn('University', regexp_replace('University', '[^A-Za-z0-9 ]+', ''))\
    .withColumn('Difficulty Level', regexp_replace('Difficulty Level', '[^A-Za-z0-9 ]+', ''))\
    .withColumn('Course Rating', regexp_replace('Course Rating', '[^A-Za-z0-9 ]+', ''))\
    .withColumn('Course Description', regexp_replace('Course Description', '[^A-Za-z0-9 ]+', ''))\
    .withColumn('Skills', regexp_replace('Skills', '[^A-Za-z0-9 ]+', ''))
data.show()

+--------------------+--------------------+----------------+--------------+--------------------+--------------------+--------------------+
|         Course Name|          University|Difficulty Level| Course Rating|          Course URL|  Course Description|              Skills|
+--------------------+--------------------+----------------+--------------+--------------------+--------------------+--------------------+
|Write A Feature L...|Michigan State Un...|        Beginner|            48|https://www.cours...|Write a Full Leng...| writing You will...|
|Business Strategy...|Coursera Project ...|        Beginner|            48|https://www.cours...|By the end of thi...|Finance  business...|
|Silicon Thin Film...|  cole Polytechnique|        Advanced|            41|https://www.cours...|This course consi...| thin film semico...|
|Finance for Managers|IESE Business School|    Intermediate|            48|https://www.cours...|When it comes to ...|accounts receivab...|
|Retrieve Data usi...|Cours

In [ ]:
# Counting and removing null values
data.select([count(when(isnull(c), c)).alias(c) for c in data.columns]).show()

+-----------+----------+----------------+-------------+----------+------------------+------+
|Course Name|University|Difficulty Level|Course Rating|Course URL|Course Description|Skills|
+-----------+----------+----------------+-------------+----------+------------------+------+
|          0|         5|               6|            6|         7|                 7|    11|
+-----------+----------+----------------+-------------+----------+------------------+------+



In [ ]:
data = data.na.drop()
data.select([count(when(isnull(c), c)).alias(c) for c in data.columns]).show()

+-----------+----------+----------------+-------------+----------+------------------+------+
|Course Name|University|Difficulty Level|Course Rating|Course URL|Course Description|Skills|
+-----------+----------+----------------+-------------+----------+------------------+------+
|          0|         0|               0|            0|         0|                 0|     0|
+-----------+----------+----------------+-------------+----------+------------------+------+



In [ ]:
# Identifying no:of unique values in each column in data
data.agg(*(countDistinct(col(c)).alias(c) for c in data.columns)).show()

+-----------+----------+----------------+-------------+----------+------------------+------+
|Course Name|University|Difficulty Level|Course Rating|Course URL|Course Description|Skills|
+-----------+----------+----------------+-------------+----------+------------------+------+
|       3414|       185|               6|           32|      3422|              3382|  3393|
+-----------+----------+----------------+-------------+----------+------------------+------+



# **Setup Metadata**

In [ ]:
# Setup metadata to reduce duplication
from pyspark.sql import Window

window = Window.orderBy(['University', 'Difficulty Level', 'Course Rating'])
data = data.withColumn('recordID',dense_rank().over(window) - 1)
data.orderBy(['University', 'Difficulty Level', 'Course Rating', 'recordID']).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|         Course Name|          University|    Difficulty Level|       Course Rating|          Course URL|  Course Description|              Skills|recordID|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|  If you have com...| signed up to the...| met all the asse...| this course is f...| but this course ...| through the Conn...| US district lead...|       0|
|The Business of P...|Advancing Women i...|            Advanced|                  48|https://www.cours...|Sponsored by AMAZ...|market segmentati...|       1|
|Acing the Product...|Advancing Women i...|            Advanced|                   5|https://www.cours...|Sponsored by AMAZ...|process managemen...|       2|
|The Art  Science ...|Advancing Women i...|         

In [ ]:
metadata = data.select("recordID").drop_duplicates()
metadata.show()

+--------+
|recordID|
+--------+
|       0|
|       1|
|       2|
|       3|
|       4|
|       5|
|       6|
|       7|
|       8|
|       9|
|      10|
|      11|
|      12|
|      13|
|      14|
|      15|
|      16|
|      17|
|      18|
|      19|
+--------+
only showing top 20 rows



# **Retrieve any record given the record ID**

In [ ]:
# Get an ID from metadata
record_ID = list(map(lambda row: row[0], metadata.rdd.takeSample(False, 1)))[0]
record_ID

1541

In [ ]:
# Retrieve records belonging to ID
result_df = data.filter(data.recordID == record_ID)
result_df.show()

+--------------------+--------------------+----------------+-------------+--------------------+--------------------+--------------------+--------+
|         Course Name|          University|Difficulty Level|Course Rating|          Course URL|  Course Description|              Skills|recordID|
+--------------------+--------------------+----------------+-------------+--------------------+--------------------+--------------------+--------+
|Lesson  Business ...|University of Was...|    Intermediate|           42|https://www.cours...|This lesson is pa...|english language ...|    1541|
+--------------------+--------------------+----------------+-------------+--------------------+--------------------+--------------------+--------+



In [ ]:
# Store result
result_df.toPandas().to_csv("/content/drive/My Drive/spark_preprocessed_data.csv")